In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE orders_bronze
AS
SELECT 
  *,
  current_timestamp() AS ingestion_time,
  _metadata.file_name AS source_file
FROM cloud_files("${source}/orders", "json", map("cloudFiles.inferColumnTypes", "true"))


In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE orders_silver
  (CONSTRAINT valid_date EXPECT (order_timestamp > "2021-01-01") ON VIOLATION FAIL UPDATE)
COMMENT "Append only orders with valid timestamps"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  timestamp(order_timestamp) AS order_timestamp,
  * EXCEPT (order_timestamp, _rescued_data)
FROM STREAM(LIVE.orders_bronze)

In [0]:
%sql
CREATE OR REFRESH MATERIALIZED VIEW orders_by_date
AS
SELECT
  order_timestamp::date AS order_date,
  count(*) AS total_daily_orders
FROM LIVE.orders_silver
GROUP BY date(order_timestamp)